The code loads the FashionMNIST dataset (both training and test sets), 
 applies transformations (conversion to tensor and normalization), 
 and display the first image of each class in both sets(train and test) using matplotlib in a 2x5 grid layout.

In [ ]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# 데이터 전처리: 이미지를 텐서로 변환하고 정규화하기 위한 transform 정의
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Adjusted for single-channel images
])

# 학습 데이터셋 다운로드
trainset = datasets.FashionMNIST('', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# 테스트 데이터셋 다운로드
testset = datasets.FashionMNIST('', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

print(f"Training set size: {len(trainset)} images")
print(f"Test set size: {len(testset)} images")

# Plot graph for the trainset
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
fig.suptitle("Sample Images from Fashion MNIST Classes")

# Define the class labels for Fashion MNIST
class_labels = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
    'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the first image from each class in the trainset
for i in range(10):
    # 각 클래스의 첫 번째 이미지 찾기
    first_idx = next(idx for idx, (_, label) in enumerate(trainset) if label == i)
    image, label = trainset[first_idx]

    # 이미지 시각화
    ax = axes[i // 5, i % 5]
    ax.imshow(image.squeeze(), cmap='gray')  # 이미지에서 배치 차원 제거
    ax.set_title(f"Train: {class_labels[label]}")
    ax.axis('off')

plt.show()

# Plot graph for the testset
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
fig.suptitle("Sample Images from Fashion MNIST Test Set")

# Visualize the first image from each class in the testset
for i in range(10):
    # 각 클래스의 첫 번째 이미지 찾기
    first_idx = next(idx for idx, (_, label) in enumerate(testset) if label == i)
    image, label = testset[first_idx]

    # 이미지 시각화
    ax = axes[i // 5, i % 5]
    ax.imshow(image.squeeze(), cmap='gray')  # 이미지에서 배치 차원 제거
    ax.set_title(f"Test: {class_labels[label]}")
    ax.axis('off')

plt.show()

Train the data

In [ ]:
from torch import nn, optim

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 28x28 images flattened into a vector
        self.fc2 = nn.Linear(128, 10)  # Output layer with 10 classes

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        x = torch.relu(self.fc1(x))  # ReLU activation for the first hidden layer
        x = self.fc2(x)  # Output layer (logits)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")


Model Evaluation
Visualize predicted images and labels

In [ ]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

predictions = []
true_labels = []
images_for_visualization = []  # Store the images from the batch for visualization

# No need to calculate gradients during evaluation
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
        total += labels.size(0)  # Total number of images
        correct += (predicted == labels).sum().item()  # Correct predictions
        predictions.extend(predicted.numpy())  # Store predictions
        true_labels.extend(labels.numpy())  # Store true labels
        images_for_visualization.extend(images.numpy())  # Store images for visualization

accuracy = 100 * correct / total  # Calculate the accuracy percentage
print(f"Test Accuracy: {accuracy:.2f}%")

# Visualization function for predicted images with true and predicted labels
def visualize_predictions(images, true_labels, predictions, num=10):
    plt.figure(figsize=(12, 6))
    for i in range(num):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].squeeze(), cmap='gray')  # Display the image
        # Display true and predicted labels under the image
        plt.title(f'True: {class_labels[true_labels[i]]}\nPred: {class_labels[predictions[i]]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize predictions from the images stored for visualization
visualize_predictions(images_for_visualization, true_labels, predictions, num=10)


 Load images from the FashionMNIST training set. 
 3 sample images for each class are save in separate folders (e.g., T-shirt_top, Trouser, etc.) 
 inside the base directory ./Fmnist_images

In [ ]:
import os

# Base directory to save the images
save_dir = './Fmnist_images'
os.makedirs(save_dir, exist_ok=True)

# Define the class labels for Fashion MNIST, replacing slashes with underscores to avoid confusion
class_labels = [
    'T-shirt_top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
    'Shirt', 'Sneaker', 'Bag', 'Ankle_boot'
]

# Dictionary to keep track of saved images per class
saved_counts = {class_name: 0 for class_name in class_labels}

# Find and save images for each class
for idx, (image_tensor, label) in enumerate(trainset):
    class_name = class_labels[label]

    # Check if we have saved 3 images for this class
    if saved_counts[class_name] < 3:
        # Create a folder for each class inside the save directory
        class_dir = os.path.join(save_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)

        # De-normalize and convert to PIL Image
        image_tensor = image_tensor * 0.5 + 0.5  # De-normalize to [0, 1] range
        image_pil = transforms.ToPILImage()(image_tensor.squeeze(0))  # Remove channel dimension

        # Save the image with a unique filename in the class folder
        image_path = os.path.join(class_dir, f"{class_name}_{saved_counts[class_name] + 1}.png")
        image_pil.save(image_path)
        print(f"Saved {image_path}")

        # Update the count for this class
        saved_counts[class_name] += 1

    # Stop if we have saved 3 images for each class
    if all(count == 3 for count in saved_counts.values()):
        break

print("Images saved successfully.")
